### **STEP-1**. 준비 작업
AI contents 실행 (python main.py) 후에 등록 과정을 진행 할 수 있습니다. !! 

jupyter kernel 을 제작합니다. 
```bash
    conda activate {ENV-NAME}           ## python main.py 실행 용 가상환경
    pip install ipykernel        
    python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}
```


----

### **STEP-2**. Solution 정보 입력 및 Infra 정보 입력

##### Solution information (dict) 작성 
- inference_only: 파이프라인이 학습 및 추론 과정을 지원하는 표시     
  - ex1: 학습 과 추론 모두 지원 --> False     
  - ex2: 추론만 지원 --> True    
  - ex3: 학습 만은 미지원 --> ~~['train']~~ 

- solution_update: 기존 솔루션을 업데이트 할지 결정
  - True: 업데이트 진행. 기존 name 과 동일한 이름 입력 (name 이 존재하지 않으면 에러)  
  - False: 신규 생성. 기존 name 과 다르게 입력 (name 이 존재하면 에러)

- solution_name: 솔루션 이름 
  - 주의사항: 스페이스, 특수 문자, 한글은 미지원 

- solution_type: AI Solution 이 공유되는 계정 범위. 'private', 'public' 중에 하나 선택.   
  - 'public' : AI-Advisor 가 설치될 때, built-in 되어 설치 할 solution 의미.   
    (SYSTEM MANAGER 만 등록 가능)
  - 'private': 과제 단위로 공유 될 solution. 

- description: 솔루션이 Edge Conductor 의 UI 로 표시 될 때, 설명 가이드 작성 
  - overview: 솔루션에 대한 전반적을 설명을 작성. (markdown 을 지원)
  - 고려사항: overview 외에 다양한 sub-title 지원 예정.

- contents_type: re-train, re-labeling 을 위한 설명 
  - support_labeling: re-labeling 할지 결정. True 일 경우,   
    Edge Conductor 에서 re-labeling 기능 활성화.   
  - inference_result_datatype: EdgeApp 에 inference 결과를 표시하는 방법으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함
  - train_datatype: re-train 에 사용될 데이터 포멧 결정으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함

- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 train docker container 제작 
- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 inference docker container 제작 
- inference_arm: True, False 중에 선택. True 일 경우, arm 용 inference docker container 제작


    

In [1]:
#----------------------------------------#
#        AI Solution Spec 작성           #
#----------------------------------------#
solution_info ={
    'inference_only': True, # True, False
    'solution_update': False,
    'solution_name': 'solution_name',
    'solution_type': "private", ##public, private 
    'description': {
        "overview": "Provide a description of the AI solution."
    },
    'contents_type': {
            'support_labeling': False,
            'inference_result_datatype': 'table', # 'image'
            'train_datatype': 'table' # 'image'
    },
    'train_gpu': False, ## cpu, gpu
    'inference_gpu': False,
    "inference_arm": False,  # amd, arm  
}

##### AI Solution 이 동작될 Infra_setup 파일 load 하기
./setting/infra_setup.yaml 를 참조하여 infra 환경을 설정합니다. 

### **STEP-3**. AI Solution 등록

##### 3-1. Username & Password 를 입력 

> !! 접속이 실패되는 경우, AI Conductor 에게 계정 생성을 요청 하시기 바랍니다.   
> Contact Us: hyunsoo0802.lim@lge.com  (LGE AI빅데이터담당, AI Conductor 담당자)

> Login 방식이 {'LOGIN_METHOD': 'ldap'} 로 설정되어 있는 경우, LGEP 의 ID & PW 를 사용


In [2]:
import getpass

username = input('Username: ')
password = getpass.getpass('Password: ')

print("Your ID : ", username)
print("Your PW : ", password.replace(password, '*' * len(password)))

Your ID :  lge-alo
Your PW :  ***********


##### 3-2. ALO 솔루션 등록 실행
아래와 같은 과정이 순차적으로 실행 됨. 진행 중 Error 가 발생하게 되면, STEP-2 설정을 변경 함.
> Contact US: sehyun.song@lge.com (LGE AI빅데이터 담당, ALO 담당자)

AI Solution 등록 과정 Process:  
> 1.  solution name 입력 
> 2.  description & wrangler code 를 solution_metadata 에 삽입
> 3.  contents_type 을 solution_metadata 에 삽입
> 4.  description 의 solution icon 을 solution_metadata 에 삽입
> 5.  train 용 user parameter 를 solution_metadata 에 삽입
> 6.  s3 에 train data & artifacts 업로드 
> 7.  train 용 ecr repository 생성 및 docker container 업로드 
> 8.  inference 용 user parameter 를 solution_metadata 에 삽입
> 9.  s3 에 inference data & artifacts 업로드 
> 10. inference 용 ecr repository 생성 및 docker container 업로드 
> 11. solution 등록 



In [6]:
import sys

try:
    del sys.modules['src.solution_register'] 
except:
    pass
from src.solution_register import SolutionRegister

infra = "./setting/infra_config.meerkat.yaml"
register = SolutionRegister(infra_setup=infra, solution_info=solution_info)

register.debugging = False  ## default: False (skip 항목: docker 생성, solution 등록)
register.skip_generation_docker = False  ## default: False
register.run(username=username, password=password)


######################################################
#######    Initiate ALO operation mode
########################################################

[SYSTEM] Solutoin 등록에 필요한 setup file 들을 load 합니다. 
Infra setup 파일을 load 합니다. (path: ./setting/infra_config.meerkat.yaml)
{'AIC_URI': 'https://meerkat-ai.com/',
 'AWS_KEY_FILE': '../aws-meerkat.key',
 'BUILD_METHOD': 'docker',
 'LOGIN_MODE': 'static',
 'REGION': 'ap-northeast-2',
 'REPOSITORY_TAGS': [],
 'SOLUTION_TYPE': 'private',
 'VERSION': 1.0,
 'WORKSPACE_NAME': 'lge-ws'}
{'contents_type': {'inference_result_datatype': 'table',
                   'support_labeling': False,
                   'train_datatype': 'table'},
 'description': {'overview': 'Provide a description of the AI solution.'},
 'inference_arm': False,
 'inference_gpu': False,
 'inference_only': True,
 'solution_name': 'solution_name',
 'solution_type': 'private',
 'solution_update': False,
 'train_gpu': False}
[SYSTEM] S3 key 파일을 로드 합니다. (file: ../aws-meerkat.key)
[

WARNING! Your password will be stored unencrypted in /home/sehyun.song/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[SYSTEM] AWS ECR | docker login result:
Login Succeeded

[SYSTEM] Target AWS ECR repository:
ecr-repo-an2-meerkat-dev/lge/ai-solutions/solution_name/inference/solution_name
[SYSTEM] AWS ECR create-repository response: 
{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-2:856124245140:repository/ecr-repo-an2-meerkat-dev/lge/ai-solutions/solution_name/inference/solution_name', 'registryId': '856124245140', 'repositoryName': 'ecr-repo-an2-meerkat-dev/lge/ai-solutions/solution_name/inference/solution_name', 'repositoryUri': '856124245140.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-meerkat-dev/lge/ai-solutions/solution_name/inference/solution_name', 'createdAt': datetime.datetime(2024, 1, 31, 17, 46, 36, 580000, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '878adb25-4b18-4a1b-93eb-5e4423575439', 'HTTPStatusCode': 200, 'HTTPHead

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.66kB done
#1 ...

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.2s

#1 [internal] load build definition from Dockerfile
#1 DONE 0.3s

#3 [internal] load metadata for public.ecr.aws/docker/library/python:3.10-slim-bullseye
#3 DONE 0.0s

#4 [ 1/12] FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 232.89kB 0.0s done
#5 DONE 0.2s

#6 [ 2/12] RUN apt-get update
#6 CACHED

#7 [ 3/12] RUN apt-get install -y apt-utils
#7 CACHED

#8 [ 4/12] RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc          docker.io          procps          jq     && rm -rf /var/lib/apt/lists/*
#8 CACHED

#9 [ 5/12] COPY /.register_source /framework/
#9 DONE 1.9s

#10 [ 6/12] COPY /solution_metadata.yaml /framework/
#10 DONE 0.8s

#11 [ 7/12

The push refers to repository [856124245140.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-meerkat-dev/lge/ai-solutions/solution_name/inference/solution_name]
25fe7fe162e5: Preparing
8d7248da3752: Preparing
f45a025aaa82: Preparing
9552ee23960e: Preparing
5f70bf18a086: Preparing
dc6d7eb89fde: Preparing
e605f53031e7: Preparing
0ff6af9334e4: Preparing
0941f5db1f5c: Preparing
4cac178e7314: Preparing
dc4295a3639b: Preparing
9647f452a529: Preparing
7972dfbb0bc8: Preparing
29578fbbec63: Preparing
cc056986b725: Preparing
74c0af6e0227: Preparing
dc4295a3639b: Waiting
9647f452a529: Waiting
7972dfbb0bc8: Waiting
29578fbbec63: Waiting
0ff6af9334e4: Waiting
cc056986b725: Waiting
74c0af6e0227: Waiting
4cac178e7314: Waiting
0941f5db1f5c: Waiting
e605f53031e7: Waiting
dc6d7eb89fde: Waiting
5f70bf18a086: Pushed
f45a025aaa82: Pushed
dc6d7eb89fde: Pushed
9552ee23960e: Pushed
e605f53031e7: Pushed
0ff6af9334e4: Pushed
4cac178e7314: Pushed
8d7248da3752: Pushed
7972dfbb0bc8: Pushed
9647f452a529: Pushed
dc

##### 등록된 AI Solution 테스트 하기  (Manager 권한만 가능만)

In [7]:
try:
    del sys.modules['src.solution_register'] 
except:
    pass
from src.solution_register import SolutionRegister


infra = "./setting/infra_config.meerkat.yaml"
register = SolutionRegister(
    infra_setup=infra, 
    solution_info=solution_info)
    
register.login(username, password)
register.run_train(
    status_period = 10, ## 몇 초 간격으로 학습 상태를 체크할 것인지 설정
    delete_instance = True,   ## 학습 테스트에 사용한 solution instance & stream 을 삭제할 것인지 설정 
    delete_solution= True,  ## solution 을 삭제할 것인지 설정
    )

NameError: name 'infra_setup' is not defined

##### 등록한 Solution List 를 확인하기 (Manager 권한만 가능)

In [ ]:
try:
    del sys.modules['src.solution_register'] 
except:
    pass

from src.solution_register import SolutionRegister

register = SolutionRegister(
    infra_setup=infra_setup, 
    solution_info=solution_info)
    
register.login(username, password)
register.list_solution()

Solution 을 삭제 합니다. 

In [ ]:
register.delete_solution()